# Sentiment analysis 

This project involves building a movie review classifier
Sentiment analysis is the task of classifying the polarity of a given text.

The dataset used is the IMDb dataset. The data was compiled by Andrew Maas (http://ai.stanford.edu/~amaas/data/sentiment/)
This dataset contains 50,000 reviews labelled as either positive or negative. Moreover, this datset is balanced, i.e. equal number of postive and negative reviews. Moreover, for each movie, a number equal or less than 30 reviews is considered since reviews tend to correlate.
Finally, only reviews with either a low score (less or equal to 4 out of 10) or high score (greater or equal to 7 out of 10) are considered to avoid considering reviews that are neutral.
For the purpose of testing, we are going to use only the training data since they are labelled. 


For this problem, we are going to use a state-of-the-art language model pre-training model, BERT (Bidirectional Encoder Representations from Transformers)model and we will apply transfer learning to adapt the model to the sentiment analysis problem. 
The model is going to be loaded from the library transformers (https://github.com/huggingface/transformers). We are going to use Pytorch as framework for this problem.

## Data processing 
For the data processing task:
- Lowercasing is applied. Lowercasing ALL the text data, is one of the simplest and most effective form of text preprocessing. Some pre-trained model might give a different answers if the input is capitalized or not. Of course, in some specific cases, it might be useful not to change to lowercasing.
- Noise Removal is applied. Noise removal is about removing characters digits and pieces of text that can interfere with your text analysis. For instances, punctuations signs, parenthesis, mathematical symbols are removed (for instance, '.' '(' '?' or '<' or ':' are removed).
- Since we are using BERT model, we keep the words as they are, i.e. stemming is **not** used. Stemming is the process of reducing inflection in words (e.g. connected, connections) to their root form (e.g. connect).
- For the same reason explained before, lemmatization is **not** used. Lemmatization is very similar to stemming. The difference is that, lemmatization doesn’t just chop things off, it actually transforms words to the actual root.

Once the data is pre-processed, then the training data (25,000 samples) are used to train the model and testing data (25,000 samples) are used to test the final model.

## Building the model
The model used is the pre-trained BERT model with a final fully connected layer. 
Specifically, it is used BERT base model – with a hidden size of 768, with 12 layers (transformer blocks), 12 attention heads, and 110 million parameters.

The input to BERT is a sequence of words, and the output is a sequence of vectors.
BERT model can accept up to 512 words. However, some review sentences might be longer than this number. In order to deal with this, we apply the strategy described in the paper "How to Fine-Tune BERT for Text Classification?" by Sun et al. They propose to use a truncation method, and select part of the beginning and part of the end of the sentence. 
In fact, the key information of an article is at the beginning and end. Thus, empirically the first 128 and the last 382 tokens are selected if longer than 512.

The neural network model will have first the 11 BERT layers, then droput and finally a linear layer. Droput is added for the purpose of Regulariaztion and the linear layer for Classification.


## Training the model
The final layer is used to compute the loss and determine the accuracy of the model. The loss used is Binary Cross Entropy which is implemented as BCELogits Loss in PyTorch.
A learning rate of 2.5e-5 is used. The optimization algorithm used is ADAM with $\beta_1 = 0.9$ and $\beta_2 = 0.999$. The batch size is 10 and the number of epochs is 5. 

## Testing the model
Once the model is trained, the algorithm is tested on the test set and reach an accuracy of 99.4% using only 5 epochs! this shows how transfer learning is important for using these models for additional tasks!


Thanks,
Gabriele Boncoraglio

Import some useful libraries for running the code

In [58]:
import re,os
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report
import pandas as pd

import numpy as np
from sklearn import metrics
# !pip install -Iv transformers==2.2.2
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

os.chdir('/content/drive/My Drive/Colab Notebooks/Sentiment Analysis')
#True if you already trained the model
trained = False

cuda


## Data processing 

In [0]:
train = []
for line in open('training_dataset.txt', 'r'):
    train.append(line.strip())

test = []
for line in open('testing_dataset.txt', 'r'):
    test.append(line.strip())

In [0]:
# Data noise removal
def preprocess_reviews(reviews):
    reviews = [re.compile("[.;:!\'?,\"()\[\]]").sub("", line.lower()) for line in reviews]
    reviews = [re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
.sub(" ", line) for line in reviews]

    return reviews

train1 = preprocess_reviews(train)
test1 = preprocess_reviews(test)
#Creating labels for dataset
#first 12500 are postive and the rest is negative
target = [1 if i < 12500 else 0 for i in range(25000)]


In [30]:
# Dealing with long sentences
# Looking average and max length of the training sentences dataset
maxLen = 0
avgLen = 0
for stri in train1:
  s = stri.split()
  maxLen = max(maxLen,len(s))
  avgLen += len(s)
avgLen /= len(train1)
print('(training) average is: ',round(avgLen))
print('(training) max length is: ',maxLen)


maxLen = 0
avgLen = 0
for stri in test1:
  s = stri.split()
  maxLen = max(maxLen,len(s))
  avgLen += len(s)
avgLen /= len(train1)
print('(testing) average is: ',round(avgLen))
print('(testing) max length is: ',maxLen)

(training) average is:  233
(training) max length is:  2473
(testing) average is:  228
(testing) max length is:  2245


In [31]:
# Visualizing some of the comments and the response to see our data
d_train = {'comment_text': train1, 'list': target}
d_test = {'comment_text': test1, 'list': target}
df_train = pd.DataFrame(data=d_train)
df_test = pd.DataFrame(data=d_train)
df_train.head()

,comment_text,list
0,bromwell high is a cartoon comedy it ran at th...,1
1,homelessness or houselessness as george carlin...,1
2,brilliant over acting by lesley ann warren bes...,1
3,this is easily the most underrated film inn th...,1
4,this is not the typical mel brooks film it was...,1


In [40]:

# Creating the dataset and dataloader for the neural network
train_size = 1.0
train_dataset=df_train.sample(frac=train_size,random_state=200).reset_index(drop=True)
validation_dataset=df_train.drop(train_dataset.index).reset_index(drop=True)


print("FULL Dataset: {}".format(df_train.shape[0]+df_test.shape[0]))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("VALIDATION Dataset: {}".format(validation_dataset.shape))
print("TEST Dataset: {}".format(df_test.shape))

FULL Dataset: 50000
TRAIN Dataset: (25000, 2)
VALIDATION Dataset: (0, 2)
TEST Dataset: (25000, 2)


## Building the model

In [0]:
# Sections of config
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 10
VALID_BATCH_SIZE = 10
TEST_BATCH_SIZE = 10
EPOCHS = 5
LEARNING_RATE = 2e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')



In [0]:
'''
PyTorch supports two different types of datasets:
- map-style datasets,
- iterable-style datasets.

        - Map-style datasets
        A map-style dataset is one that implements the __getitem__() and __len__() protocols, 
        and represents a map from (possibly non-integral) indices/keys to data samples.
        For example, such a dataset, when accessed with dataset[idx], 
        could read the idx-th image and its corresponding label from a folder on the disk.
        
        - An iterable-style dataset is an instance of a subclass of IterableDataset that 
        implements the __iter__() protocol, and represents an iterable over data samples. 
        This type of datasets is particularly suitable for cases where random reads 
        are expensive or even improbable, and where the batch size depends on the fetched data.
        For example, such a dataset, when called iter(dataset), 
        could return a stream of data reading from a database, a remote server, 
        or even logs generated in real time.
'''

class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = dataframe.comment_text
        self.targets = self.data.list
        self.max_len = max_len


    def __len__(self):
        return len(self.comment_text)
    
 
    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())
        if len(comment_text) > 512:
          comment_text = comment_text[:128] + comment_text[381:] 

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            stride = 0,
            truncation_strategy = 'longest_first',
            pad_to_max_length = True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [0]:
# Creating a Map-style datasets for PyTorch
training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
validation_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(df_test, tokenizer, MAX_LEN)

In [0]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

                }
validation_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0

training_loader = DataLoader(training_set, **train_params)
validation_loader = DataLoader(validation_set, **validation_params)
testing_loader = DataLoader(testing_set, **test_params)

In [45]:
'''
    We will be creating a neural network with the BERTClass.
    This network will have the Bert model. Follwed by a Droput and Linear Layer. 
    They are added for the purpose of Regulariaztion and Classification respectively.
    
    Final layer outputs is what will be used to calcuate the loss and 
    to determine the accuracy of models prediction.
    The loss function used will be a combination of Binary Cross Entropy 
    which is implemented as BCELogits Loss in PyTorch
'''
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        '''
            In the forward loop, there are 2 output from the BertModel layer.
            The second output output_1 or called the pooled output is passed to the Drop Out layer and 
            the subsequent output is given to the Linear layer.
            Keep note the number of dimensions for Linear Layer is 6 because that is 
            the total number of categories in which we are looking to classify our model.
        '''
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        # #Freeze bert layers
        # for p in self.l1.parameters():
        #     p.requires_grad = False
        self.l2 = torch.nn.Dropout(0.1)
        self.l3 = torch.nn.Linear(768, 1,bias=True)
        # self.l4 = torch.nn.Tanh()
        # self.l5 = torch.nn.Linear(100, 6,bias=True)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        # output_4 = self.l4(output_3)
        # output = self.l5(output_4)
        return output

model = BERTClass()

if trained:
  model.load_state_dict(torch.load("bert_sentimentAnaly"))
model.to(device)



BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [0]:
import time
import datetime

def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)


optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE, weight_decay = 5e-4)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

def validation(loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            targets = targets.unsqueeze(1)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

def computeScores(testing_loader,typeSet):
    outputs, targets = validation(testing_loader) 
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print("{}".format(typeSet))
    print(f"  Accuracy Score = {accuracy}")
    print(f"  F1 Score (Micro) = {f1_score_micro}")
    print(f"  F1 Score (Macro) = {f1_score_macro}")
    print(" ")
    return outputs, targets
    # return accuracy,f1_score_micro,f1_score_macro

In [51]:
if not trained:
  for epoch in range(EPOCHS):
      # ========================================
      #               Training
      # ========================================
      
      # Perform one full pass over the training set.

      print("")
      print('======== Epoch {:} / {:} ========'.format(epoch + 1, EPOCHS))
      print('Training...')
      # Measure how long the training epoch takes.
      t0 = time.time()

      # Reset the total loss for this epoch.
      total_train_loss = 0

      # Put the model into training mode. Don't be mislead--the call to 
      # `train` just changes the *mode*, it doesn't *perform* the training.
      # `dropout` and `batchnorm` layers behave differently during training
      # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
      model.train()

      # for _,data in enumerate(training_loader, 0):
      for step, data in enumerate(training_loader):

          ids = data['ids'].to(device, dtype = torch.long)
          mask = data['mask'].to(device, dtype = torch.long)
          token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
          targets = data['targets'].to(device, dtype = torch.float)
          targets = targets.unsqueeze(1)
          # target = target.float()
          outputs = model(ids, mask, token_type_ids)


          # Always clear any previously calculated gradients before performing a
          # backward pass. PyTorch doesn't do this automatically because 
          # accumulating the gradients is "convenient while training RNNs". 
          # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
          optimizer.zero_grad()
          loss = loss_fn(outputs, targets)

          # Progress update every 100 batches.
          if step % 100 == 0 and not step == 0:
              # Calculate elapsed time in minutes.
              elapsed = format_time(time.time() - t0)
              
              # Report progress.
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.      Current Loss: {}'.format(step, len(training_loader), elapsed,loss.item()))

          # Accumulate the training loss over all of the batches so that we can
          # calculate the average loss at the end. `loss` is a Tensor containing a
          # single value; the `.item()` function just returns the Python value 
          # from the tensor.
          total_train_loss += loss.item()

          
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

      # Calculate the average loss over all of the batches.
      avg_train_loss = total_train_loss / len(training_loader)            
      
      # Measure how long this epoch took.
      training_time = format_time(time.time() - t0)

      print("")
      print("  Average training loss: {0:.2f}".format(avg_train_loss))
      print("  Training epcoh took: {:}".format(training_time))
      print("")
      # computeScores(testing_loader,'Testing set')
      # computeScores(validating_loader,'Validation set')
      # computeScores(validating_loader,'Validation set')
      print("")

if not trained:
  torch.save(model.state_dict(), "bert_sentimentAnaly")


======== Epoch 1 / 5 ========
Training...
  Batch   100  of  2,500.    Elapsed: 0:01:03.      Current Loss: 0.6109544038772583
  Batch   200  of  2,500.    Elapsed: 0:02:06.      Current Loss: 0.5833436846733093
  Batch   300  of  2,500.    Elapsed: 0:03:09.      Current Loss: 0.05922584608197212
  Batch   400  of  2,500.    Elapsed: 0:04:12.      Current Loss: 0.2766300141811371
  Batch   500  of  2,500.    Elapsed: 0:05:15.      Current Loss: 0.4598260819911957
  Batch   600  of  2,500.    Elapsed: 0:06:18.      Current Loss: 0.4994557797908783
  Batch   700  of  2,500.    Elapsed: 0:07:21.      Current Loss: 0.033208929002285004
  Batch   800  of  2,500.    Elapsed: 0:08:24.      Current Loss: 0.29058030247688293
  Batch   900  of  2,500.    Elapsed: 0:09:27.      Current Loss: 0.1628505438566208
  Batch 1,000  of  2,500.    Elapsed: 0:10:29.      Current Loss: 0.3567003011703491
  Batch 1,100  of  2,500.    Elapsed: 0:11:32.      Current Loss: 0.3144363462924957
  Batch 1,200  of 

In [55]:
predictions, targets = computeScores(testing_loader,'Testing set')

Testing set
  Accuracy Score = 0.99484
  F1 Score (Micro) = 0.99484
  F1 Score (Macro) = 0.9948399431237891
 


In [59]:
roc_auc_score(targets, predictions)

0.99484

In [60]:
# classification_report(y_true, y_pred)
print(classification_report(targets, predictions))

              precision    recall  f1-score   support

         0.0       1.00      0.99      0.99     12500
         1.0       0.99      1.00      0.99     12500

    accuracy                           0.99     25000
   macro avg       0.99      0.99      0.99     25000
weighted avg       0.99      0.99      0.99     25000

